# Libraries

In [1]:
import os
import xarray as xr
import rioxarray as rxr

# Directories

In [33]:
main_dir = r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\VCI\NDVI-16\Trial\Tiles\Crop_mask_new'
# Open the Phenology image which contains 4 bands of start and end of
# first and second seasons downloaded from ASAP-Global land surface phenology
Phenology = rxr.open_rasterio(main_dir  +'\\' + 'Phenology.tif',masked=True)
# Resampled (to 300m) over South Africa of 
# CCI LAND COVER - S2 PROTOTYPE LAND COVER 20M MAP OF AFRICA 2016
Land_cover_CCI = rxr.open_rasterio(main_dir + '\\' +'LC_300m.tif', masked = True)
# Land cover of Sentinel3 at 300m resolution over South Africa
Land_cover_S3 = rxr.open_rasterio(main_dir + '\\' +'LC_South Africa300.tif', masked = True)
# Reprojection of Land cover of Sentinel3 to match Land_cover_CCI
Land_cover_S3 = Land_cover_S3.rio.reproject_match(Land_cover_CCI)

Tiff files of phenology containg value V which is expressed over a 3 years period, from 1 to 108 dekads (i.e. ten day periods). This is done to be able to treat the circular variable dekad as an ordinal variable. To get the timing T in the 1-36 dekad units (calendar year): i)if value V is smaller or eaual with 36, then T=V, ii) if value V is larger than 36 and smaller or equal with 72, then T= V-36,and iii)if value V is larger than 72, then T=V-72.

In [9]:
# This code is to make all values to be between 0 and 36 
# which express each dekad during the year
Phenology_new = xr.where((Phenology > 36) & (Phenology <= 72), Phenology - 36, Phenology)
Phenology_new = xr.where(Phenology_new> 72 , Phenology_new - 72,Phenology_new)

C:\Users\enhi\Anaconda3\envs\rasterio\Lib\site-packages\rioxarray\_io.py:428: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out = riods.read(band_key, window=window, masked=self.masked)


In [11]:
# Accumlating the dekads to months
count = 0
P = Phenology_new
for i in range(0,12):
    # Example for the first iteration:
    # when start or end of seasons are from 1 to 3, replace them by 1 
    # which means January
    P = xr.where((Phenology_new > (0 + count)) & (Phenology_new <= (3 +count)), 1+i, P)
    count = count+3
    

In [ ]:
# List of months to be used for the names of files
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
          'October', 'November', 'December']
for month in range(0,12):
    # Bitwise operations to determine if the pixel is on the growing season
    # The growing season is all months between start and end of season
    # The output for a pixel is 1 if the month is in  growing season and 0 if not.
    Growing_season = (xr.where(
        # This part for the first growing season as P[0] represents the start of the first season
        # while P[1] represents the end of the first season
        ( (P[0]==month+1) | (P[1] ==month+1) | ((P[0]>month+1) & (P[1]<P[0]) 
                    & (P[1]>month+1)) | ((P[0]<month+1) & (P[1]>month+1)) )
        # This part for the second growing season as P[2] represents the start of the second season
        # while P[3] represents the end of the second season
                          |((P[2]==month+1) | (P[3] ==month+1) | ((P[2]>month+1) & (P[3]<P[2]) 
                    & (P[3]>month+1)) | ((P[2]<month+1) & (P[3]>month+1))) , 1,0))
    # Crop maks when the month is in the growing season and the land cover is crops based on 
    # Land_cover_S3 and Land_cover_CCI
    Crop_mask = xr.where((Growing_season == 1) & ( ((Land_cover_S3>=11) & (Land_cover_S3<=30)) | (Land_cover_CCI==4) ),1,0)
    # Nan values if it is 0 (mask out these values)
    Crop_mask = Crop_mask.where(Crop_mask!=0)
    # Transpose the xarray to be consistent with exporting function
    Crop_mask = Crop_mask.transpose('band', 'y', 'x')
    # Put CRS for the crop mask
    Crop_mask= Crop_mask.rio.write_crs(Land_cover_CCI.rio.crs)
    # Exporting crop mask as tiff file for each month
    Crop_mask.rio.to_raster(main_dir+'\\'+ 'Crop_mask'+'\\'+ months[month]+ '.tif')